# Data Extract and upload for NADAbase

## Steps

1. Extract Episodes for the period from Communicare
2. Extract ATOM data
3. Match Assessments with Episodes
4. Generate the upload Survey.txt file.

In [1]:
import os
import pandas as pd
import numpy as np
import mylogger
# from nada_load import load_and_parse_csv
from data_config import EstablishmentID_Program
from utils.df_xtrct_prep import extract_prep_data

logger = mylogger.get(__name__)
# # List of column names in the CSV
# column_names = ['ESTABLISHMENT IDENTIFIER', 'GEOGRAPHICAL LOCATION', 'PMSEpisodeID', 'PMSPersonID', 'DOB', 'DOB STATUS', 'SEX', 'COUNTRY OF BIRTH', 'INDIGENOUS STATUS', 'PREFERRED LANGUAGE', 'SOURCE OF INCOME', 'LIVING ARRANGEMENT', 'USUAL ACCOMMODATION', 'CLIENT TYPE', 'PRINCIPAL DRUG OF CONCERN', 'PDCSubstanceOfConcern', 'ILLICIT USE', 'METHOD OF USE PRINCIPAL DRUG', 'INJECTING DRUG USE', 'SETTING', 'CommencementDate', 'POSTCODE', 'SOURCE OF REFERRAL', 'MAIN SERVICE', 'EndDate', 'END REASON', 'REFERRAL TO ANOTHER SERVICE', 'FAMILY NAME', 'GIVEN NAME', 'MIDDLE NAME', 'TITLE', 'SLK', 'MEDICARE NUMBER', 'PROPERTY NAME', 'UNIT FLAT NUMBER', 'STREET NUMBER', 'STREET NAME', 'SUBURB']




In [2]:
# Global variables
ep_source_filename = 'DATS_Annual_AllPrograms_FY22-23'
ep_datasource_file_path = f"./data/in/NSW_CSV/{ep_source_filename}.csv"

extract_start_date = 20170101
extract_end_date = 20230630

fname = f"{extract_start_date}_{extract_end_date}" # TODO :forNaada

active_clients_start_date ='2022-07-01' 
active_clients_end_date = '2023-06-30'

results_folder = "./data/out/"

In [3]:

# # List of columns we care about
columns_of_interest = ['ESTABLISHMENT IDENTIFIER', 'GEOGRAPHICAL LOCATION', 'EPISODE ID','PERSON ID', 'SPECIFY DRUG OF CONCERN', 'START DATE', 'END DATE', 'SLK']
rename_columns = {
    'SPECIFY DRUG OF CONCERN': 'PDCSubstanceOfConcern',
    'START DATE': 'CommencementDate',
    'END DATE': 'EndDate',
    'EPISODE ID': 'PMSEpisodeID',
    'PERSON ID': 'PMSPersonID',
}
def convert_date(date):
    if date == '00000000':
        return None
    else:
        return pd.to_datetime(date,  format='%d%m%Y')
        # return pd.to_datetime(date,  dayfirst=True)
    
def load_and_parse_csv(filepath):
    # Load the CSV
    vectorized_convert_date = np.vectorize(convert_date)

    df = pd.read_csv(filepath,  usecols=columns_of_interest
                     , date_parser=vectorized_convert_date, parse_dates=['START DATE', 'END DATE'])
    # Apply the convert_date function to the date columns
    # df['START DATE'] = df['START DATE'].apply(convert_date)
    # df['END DATE'] = df['END DATE'].apply(convert_date)    
    df.rename(columns=rename_columns, inplace=True)
   

    # df['CommencementDate'] = pd.to_datetime(df['CommencementDate'], format='%d%m%Y')
    # df['EndDate'] = pd.to_datetime(df['EndDate'], format='%d%m%Y')    
    return df


In [4]:

ep_df  = load_and_parse_csv(ep_datasource_file_path)
ep_df['Program'] = ep_df['ESTABLISHMENT IDENTIFIER'].map(EstablishmentID_Program)

C:\Users\aftab.jalal\AppData\Local\Temp\ipykernel_4328\384636031.py:21: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(filepath,  usecols=columns_of_interest


In [5]:
len(ep_df), min(ep_df.CommencementDate), max(ep_df.CommencementDate)

(1302, Timestamp('2017-01-18 00:00:00'), Timestamp('2023-06-30 00:00:00'))

In [6]:
ep_df['Program'] = ep_df['ESTABLISHMENT IDENTIFIER'].map(EstablishmentID_Program)
ep_df.drop(columns=['ESTABLISHMENT IDENTIFIER'], inplace=True)

In [35]:
ep_df.columns

Index(['GEOGRAPHICAL LOCATION', 'PMSEpisodeID', 'PMSPersonID',
       'PDCSubstanceOfConcern', 'CommencementDate', 'EndDate', 'SLK',
       'Program'],
      dtype='object')

In [7]:
processed_df = extract_prep_data(extract_start_date, extract_end_date
                                 , active_clients_start_date
                                 , active_clients_end_date
                                 , fname)

2023-08-12 11:13:53,429 - utils.df_xtrct_prep - INFO - No processed data found, loading from raw data.
2023-08-12 11:13:53,429 - utils.io - INFO - No cached data found, loading from DB
2023-08-12 11:14:11,530 - data_prep - DEBUG - prep_dataframe of length 10285 : 
2023-08-12 11:14:11,531 - data_prep - DEBUG - 	 get_surveydata_expanded
2023-08-12 11:14:13,852 - utils.dtypes - DEBUG - convert_dtypes
2023-08-12 11:14:14,052 - utils.dtypes - INFO - fixing PDCMethodOfUse Ingests to Ingest
2023-08-12 11:14:14,181 - utils.dtypes - INFO - fixing PDCMethodOfUse Injects to Inject
2023-08-12 11:14:14,282 - utils.dtypes - INFO - fixing PDCMethodOfUse Smokes to Smoke
2023-08-12 11:14:14,384 - utils.dtypes - INFO - fixing Past4WkDailyLivingImpacted Once or twice a week to Once or twice per week
2023-08-12 11:14:14,562 - utils.dtypes - INFO - fixing Past4WkDailyLivingImpacted Three or four times a week to Three or four times per week
2023-08-12 11:14:14,727 - utils.dtypes - INFO - fixing DoYouFeelSaf

In [8]:
atom_df  = processed_df.copy()
atom_df

,SLK,RowKey,Program,SurveyName,AssessmentDate,Anyodc,Past4WkNumInjectingDays,HaveYouEverSharedEquipment,Past4WkDailyLivingImpacted,Past4WkHowOftenPhysicalHealthCausedProblems,...,SubstanceUse,K10CompletedBy,HONOSAssessment,PDCSubstanceOrGambling,PDCDaysInLast28,PDCMethodOfUse,PDCHowMuchPerOccasion,PDCUnits,PDCAgeLastUsed,PDCGoals
8193,RGNLI170319801,GOLBICE_INAS_20170111,GOLBICE,ATOM Initial Assessment,2017-01-11,no,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"Amphetamines, n.f.d",5.0,Inhale (vapour),2.0,NaN,NaN,NaN
5525,LEIES220719911,MURMICE_INAS_20170112,MURMICE,ATOM Initial Assessment,2017-01-12,no,NaN,No,NaN,NaN,...,NaN,NaN,NaN,Heroin,NaN,Inject,NaN,NaN,NaN,NaN
5246,ITLAT020619801,GOLBICE_INAS_20170113,GOLBICE,ATOM Initial Assessment,2017-01-13,no,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"Amphetamines, n.f.d",20.0,Smoke,3.0,NaN,NaN,NaN
4747,ILIIM030919891,MURMICE_INAS_20170120,MURMICE,ATOM Initial Assessment,2017-01-20,no,NaN,Yes,NaN,NaN,...,NaN,NaN,NaN,"Amphetamines, n.f.d",14.0,Inject,2.0,NaN,NaN,NaN
3424,EOGAB031019821,MURMICE_INAS_20170123,MURMICE,ATOM Initial Assessment,2017-01-23,no,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"Amphetamines, n.f.d",23.0,Smoke,6.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4460,IDALE210519931,COCO_SITK_20230629,COCO,Arcadia House Assessments,2023-06-29,yes,NaN,No,Daily or almost daily,Daily or almost daily,...,NaN,NaN,NaN,Ethanol,28.0,Ingest,34.5,standard drinks,NaN,Cease Use
4478,IDEUY140119691,SAPPHIRE_ITSP_20230629,SAPPHIRE,ATOM ITSP Review Assessment,2023-06-29,no,NaN,No,Three or four times per week,Daily or almost daily,...,NaN,NaN,NaN,Ethanol,18.0,Ingest,34.5,standard drinks,54.0,Reduce Use
8202,RIBRA260319822,MURMICE_INAS_20230629,MURMICE,ATOM Initial Assessment,2023-06-29,no,2,No,Not at all,Three or four times per week,...,NaN,NaN,NaN,Methamphetamine,2.0,Inject,1.0,points,41.0,Cease Use
7249,OSEEB160219752,MURMPP_ITSP_20230629,MURMPP,ATOM ITSP Review Assessment,2023-06-29,no,NaN,NaN,Not at all,Not at all,...,NaN,NaN,NaN,"Cannabinoids and Related Drugs, n.f.d.",2.0,Smoke,3.0,cones / joints,47.0,Cease Use


In [ ]:
atom_df = atom_df.loc[ (atom_df.AssessmentDate>= active_clients_start_date ) & (atom_df.AssessmentDate <= active_clients_end_date)]

In [21]:
# Ensure that AssessmentDate is in datetime format
atom_df['AssessmentDate'] = pd.to_datetime(atom_df['AssessmentDate'], errors='coerce')

C:\Users\aftab.jalal\AppData\Local\Temp\ipykernel_4328\1676992055.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atom_df['AssessmentDate'] = pd.to_datetime(atom_df['AssessmentDate'], errors='coerce')


In [23]:
# atom_df.drop(atom_df[atom_df.Program.isin(['TSS', 'ARCA', 'COCO', 'PSYNSW' ])].index, inplace=True)
non_nsw_programs = ['TSS', 'ARCA', 'COCO', 'PSYNSW' ]
atom_df = atom_df[~atom_df['Program'].isin(non_nsw_programs)] 


In [24]:
len(ep_df), len(atom_df) 

(1302, 1576)

In [25]:
atom_df.Program.value_counts()

Program
EUROPATH          313
GOLBICE           229
MURMICE           206
GOLBGNRL          197
SAPPHIRE          177
BEGAPATH          145
MURMPP            115
MURMWIO           105
MONPATH            55
ALONGSIDE          21
MURMHEAD           13
TSS                 0
COCO                0
ARCA                0
PSYNSW              0
SO2LI131219633      0
Name: count, dtype: int64

In [30]:
def match_assessments(episodes_df, atoms_df): #pdc_substance_mapping
    # Apply the mapping to the ESTABLISHMENT IDENTIFIER and PDCSubstanceOfConcern columns in episodes_df
    # episodes_df['ESTABLISHMENT IDENTIFIER'] = episodes_df['ESTABLISHMENT IDENTIFIER'].map(establishment_program_mapping)
    # episodes_df['PDCSubstanceOfConcern'] = episodes_df['PDCSubstanceOfConcern'].map(pdc_substance_mapping)

    # Merge the dataframes on SLK and Program
    merged_df = pd.merge(episodes_df, atoms_df, how='inner', left_on=['SLK', 'Program'], right_on=['SLK', 'Program'])

    # Filter rows where AssessmentDate falls within CommencementDate and EndDate (or after CommencementDate if EndDate is NaN)
    matched_df = merged_df.loc[((merged_df['AssessmentDate'] >= merged_df['CommencementDate']) & 
                                (merged_df['AssessmentDate'] <= merged_df['EndDate'])) |
                               ((merged_df['AssessmentDate'] >= merged_df['CommencementDate']) & 
                                (merged_df['EndDate'].isna()))]

    # Check if PDCSubstanceOfConcern matches
    # mismatched_df = matched_df.loc[matched_df['PDCSubstanceOfConcern_x'] != matched_df['PDCSubstanceOfConcern_y']]

    # if len(mismatched_df) > 0:
    #     logger.info(f"There are {len(mismatched_df)} rows where PDCSubstanceOfConcern does not match.")
    #     logger.info(mismatched_df)

    return matched_df

In [31]:
matched_df = match_assessments(ep_df, atom_df)

In [32]:
matched_df

,GEOGRAPHICAL LOCATION,PMSEpisodeID,PMSPersonID,PDCSubstanceOfConcern,CommencementDate,EndDate,SLK,Program,RowKey,SurveyName,...,SubstanceUse,K10CompletedBy,HONOSAssessment,PDCSubstanceOrGambling,PDCDaysInLast28,PDCMethodOfUse,PDCHowMuchPerOccasion,PDCUnits,PDCAgeLastUsed,PDCGoals
0,10550,1852,170,Alcohol,2021-03-19,NaT,USEUK190519821,SAPPHIRE,SAPPHIRE_ITSP_20220704,ATOM ITSP Review Assessment,...,NaN,NaN,NaN,"Cannabinoids and Related Drugs, n.f.d.",28.0,Smoke,0.5,grams,39.0,Reduce Use
1,10550,1852,170,Alcohol,2021-03-19,NaT,USEUK190519821,SAPPHIRE,SAPPHIRE_ITSP_20221110,ATOM ITSP Review Assessment,...,NaN,NaN,NaN,"Cannabinoids and Related Drugs, n.f.d.",28.0,Smoke,1.0,grams,39.0,Reduce Use
2,10550,1852,170,Alcohol,2021-03-19,NaT,USEUK190519821,SAPPHIRE,SAPPHIRE_ITSP_20230124,ATOM ITSP Review Assessment,...,NaN,NaN,NaN,"Cannabinoids and Related Drugs, n.f.d.",28.0,Smoke,1.0,grams,39.0,Cease Use
3,10550,1852,170,Alcohol,2021-03-19,NaT,USEUK190519821,SAPPHIRE,SAPPHIRE_ITSP_20230321,ATOM ITSP Review Assessment,...,NaN,NaN,NaN,"Cannabinoids and Related Drugs, n.f.d.",28.0,Smoke,2.0,grams,39.0,Cease Use
4,10550,1852,170,Alcohol,2021-03-19,NaT,USEUK190519821,SAPPHIRE,SAPPHIRE_ITSP_20230613,ATOM ITSP Review Assessment,...,NaN,NaN,NaN,"Cannabinoids and Related Drugs, n.f.d.",28.0,Smoke,2.0,grams,39.0,Cease Use
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1583,17751,73940,1561,Methamphetamine,2023-05-10,2023-05-10,OS2OS170219951,MURMPP,MURMPP_INAS_20230510,ATOM Initial Assessment,...,NaN,NaN,NaN,"Amphetamines, n.f.d.",0.0,Smoke,3.0,points,27.0,Maintain Abstinence
1584,17751,74386,9517,Alcohol,2023-05-15,NaT,ASIHA100719861,MURMPP,MURMPP_INAS_20230515,ATOM Initial Assessment,...,NaN,NaN,NaN,Ethanol,0.0,Ingest,0.0,standard drinks,36.0,Maintain Abstinence
1585,17751,74654,9522,Alcohol,2023-05-17,NaT,ARHAI270720002,MURMPP,MURMPP_INAS_20230517,ATOM Initial Assessment,...,NaN,NaN,NaN,Ethanol,16.0,Ingest,17.0,standard drinks,22.0,Reduce Harms
1586,17751,77768,9598,Alcohol,2023-06-19,NaT,OY2AM021219991,MURMPP,MURMPP_INAS_20230619,ATOM Initial Assessment,...,NaN,NaN,NaN,Ethanol,12.0,Ingest,17.0,standard drinks,23.0,Reduce Harms


In [40]:
# Since the primary key in episodes_df is PMSEpisodeID, and in atoms_df it is a combination of SLK and Program, 
# you can use the groupby method on the merged_df to group by SLK and Program, 
# and then check if there is more than one unique PMSEpisodeID for any group.

# grouped = matched_df.groupby(['SLK', 'Program'])['PMSEpisodeID'].nunique()
grouped = matched_df.groupby(['SLK', 'RowKey'])['PMSEpisodeID'].nunique()
duplicates = grouped[grouped > 1]


In [41]:
duplicates

Series([], Name: PMSEpisodeID, dtype: int64)

In [38]:
# SLK + Program
onedupe = matched_df.loc[(matched_df.SLK =='ACARO150719751') & (matched_df.Program == 'EUROPATH')]

In [39]:
# onedupe [['Program', 'SLK', 'AssessmentDate', 'PMSEpisodeID', 'RowKey']]

,Program,SLK,AssessmentDate,PMSEpisodeID,RowKey
524,EUROPATH,ACARO150719751,2022-09-13,52138,EUROPATH_INAS_20220913
528,EUROPATH,ACARO150719751,2023-01-20,63634,EUROPATH_INAS_20230120
532,EUROPATH,ACARO150719751,2023-06-08,76935,EUROPATH_INAS_20230608


In [42]:
matched_df['Program'].value_counts()

Program
EUROPATH    259
GOLBICE     208
GOLBGNRL    181
MURMICE     161
SAPPHIRE    155
BEGAPATH    127
MURMPP       94
MURMWIO      82
MONPATH      44
MURMHEAD      7
Name: count, dtype: int64

In [43]:
# matched_df.to_csv(f"{results_folder}matched_df.csv", index=False)

In [44]:
matched_df.PMSEpisodeID.unique().shape

(823,)

In [45]:
atom_df[ (atom_df.AssessmentDate >= active_clients_start_date ) & (atom_df.AssessmentDate <= active_clients_end_date)].count()

SLK                      1576
RowKey                   1576
Program                  1576
SurveyName               1576
AssessmentDate           1576
                         ... 
PDCMethodOfUse           1576
PDCHowMuchPerOccasion    1507
PDCUnits                 1495
PDCAgeLastUsed           1534
PDCGoals                 1569
Length: 186, dtype: int64

In [46]:
len(atom_df[ (atom_df.AssessmentDate >= active_clients_start_date ) & (atom_df.AssessmentDate <= active_clients_end_date)])

1576

In [47]:
len(ep_df)

1302